In [1]:
# Import library yang diperlukan
# pandas: manipulasi data
# kagglehub: helper untuk mengunduh dataset dari Kaggle tanpa kredensial file lokal
# xgboost: model Gradient Boosted Trees yang mendukung fitur kategorikal dan nilai hilang
# sklearn: utilitas pembagian data dan evaluasi
import pandas as pd
import kagglehub
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [2]:
# Unduh dataset Titanic terbaru dari Kaggle menggunakan kagglehub
# Catatan: Pastikan internet aktif. File akan disimpan di cache lokal kagglehub
path = kagglehub.dataset_download("yasserh/titanic-dataset")

# Tampilkan lokasi file yang diunduh untuk referensi
print("Path to dataset files:", path)

# Baca file CSV ke DataFrame pandas
# Nama file utama pada dataset: Titanic-Dataset.csv
df = pd.read_csv(f"{path}/Titanic-Dataset.csv")

# Lihat beberapa baris teratas untuk memastikan pembacaan berhasil
df.head()

100%|██████████| 22.0k/22.0k [00:00<00:00, 1.05MB/s]

Extracting model files...
Path to dataset files: C:\Users\barak\.cache\kagglehub\datasets\yasserh\titanic-dataset\versions\1


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
# Hapus kolom yang tidak diperlukan untuk prediksi
df.drop(columns=['PassengerId', 'Name', 'Ticket', 'Cabin'], inplace=True)

# Konversi kolom bertipe object menjadi category (untuk dukungan kategorikal native XGBoost)
for col in df.select_dtypes(include=['object']).columns:
    df[col] = df[col].astype('category')

In [4]:
# Periksa jumlah nilai hilang (missing values) per kolom
# Tidak perlu menangani sekarang; akan kita biarkan untuk ditangani oleh model XGBoost
df.isna().sum()

Survived      0
Pclass        0
Sex           0
Age         177
SibSp         0
Parch         0
Fare          0
Embarked      2
dtype: int64

In [5]:
# Lihat ringkasan informasi dataset (tipe data, non-null count, dll)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   Survived  891 non-null    int64   
 1   Pclass    891 non-null    int64   
 2   Sex       891 non-null    category
 3   Age       714 non-null    float64 
 4   SibSp     891 non-null    int64   
 5   Parch     891 non-null    int64   
 6   Fare      891 non-null    float64 
 7   Embarked  889 non-null    category
dtypes: category(2), float64(2), int64(4)
memory usage: 43.9 KB


Catatan: Dataset memiliki missing values dan fitur kategorikal. Kita mengonversi kolom bertipe object menjadi pandas Categorical agar XGBoost dapat menangani fitur kategorikal secara native. Dengan DMatrix dan `enable_categorical=True`, nilai hilang juga akan ditangani otomatis.

In [6]:
# Siapkan fitur (X) dan target (y)
y = df['Survived']
X = df.drop(columns=['Survived'])

# Bagi data menjadi train/test
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Ubah dataset menjadi struktur data teroptimasi bernama DMatrix yang didukung oleh XGBoost.
dtrain = xgb.DMatrix(X_train, label=y_train, enable_categorical=True)
dtest = xgb.DMatrix(X_test, label=y_test, enable_categorical=True)

# Parameter training XGBoost
params = {
    'objective': 'binary:logistic',
    'eval_metric': 'logloss',
    'eta': 0.1,
    'max_depth': 5,
    'subsample': 0.9,
    'colsample_bytree': 0.9,
    'seed': 42,
}
num_boost_round = 300

# Latih model
bst = xgb.train(params, dtrain, num_boost_round=num_boost_round)

# Prediksi probabilitas dan hitung akurasi dengan threshold 0.5
y_prob = bst.predict(dtest)
y_pred = (y_prob >= 0.5).astype(int)
print(f"Akurasi: {accuracy_score(y_test, y_pred):.4f}")

Akurasi: 0.7989
